# Python for Scientists
## Day 2 Notebook
This notebook was developed for the NASA Goddard Space Flight Center and CRESST II's post-baccalaureate program by [Joe P. Renaud](https://www.josephrenaud.com) in Spring, 2024.

**Prerequisites for running this notebook**
- All the pre-reqs for Day 1 plus...
- Open val1 terminal and install packages using pip (or `conda install`):
    - `pip install astropy pytest numba`

## Improving Python's Performance
### How to test performance

In [1]:
import numpy as np

def my_func(an_array):
    return np.abs(an_array)

x = np.linspace(-10., 10., 100, dtype=np.float64)

print(my_func(x))

[10.          9.7979798   9.5959596   9.39393939  9.19191919  8.98989899
  8.78787879  8.58585859  8.38383838  8.18181818  7.97979798  7.77777778
  7.57575758  7.37373737  7.17171717  6.96969697  6.76767677  6.56565657
  6.36363636  6.16161616  5.95959596  5.75757576  5.55555556  5.35353535
  5.15151515  4.94949495  4.74747475  4.54545455  4.34343434  4.14141414
  3.93939394  3.73737374  3.53535354  3.33333333  3.13131313  2.92929293
  2.72727273  2.52525253  2.32323232  2.12121212  1.91919192  1.71717172
  1.51515152  1.31313131  1.11111111  0.90909091  0.70707071  0.50505051
  0.3030303   0.1010101   0.1010101   0.3030303   0.50505051  0.70707071
  0.90909091  1.11111111  1.31313131  1.51515152  1.71717172  1.91919192
  2.12121212  2.32323232  2.52525253  2.72727273  2.92929293  3.13131313
  3.33333333  3.53535354  3.73737374  3.93939394  4.14141414  4.34343434
  4.54545455  4.74747475  4.94949495  5.15151515  5.35353535  5.55555556
  5.75757576  5.95959596  6.16161616  6.36363636  6

In [8]:
# All this function does is "wrap" a numpy function. Meaning that `my_func` should always be slower than just calling `np.abs` by itself
# Let's see if that is true using some Jupyter Notebook "magic"

# Our function
%timeit my_func(x)

# Numpy's standalone
%timeit np.abs(x)

# Depending on your computer you should see that our function is slightly slower than pure numpy

785 ns ± 6.01 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)
735 ns ± 2.02 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Creating high-performance functions
Python is classically called a "slow" language. There is truth to that and lots of reasons why Python is slower than languages like C or Fortran.
But there are tricks we can use to greatly improve the speed of Python with very little work on our part. And never needing learn and write C.

In [2]:
# First let's make a function that we can use to compare against. This will work on a 3D numpy array.

def python_func(array_3d):

    # Get the size of each dimension
    x_len, y_len, z_len = array_3d.shape

    output = 0
    for i in range(x_len):
        for j in range(y_len):
            for k in range(z_len):

                # Output is a sum of each member of the array minus that item's x-location
                output += array_3d[i, j, k] - i
    return output

# Let's make a large random 3D array to test this on
test_array = np.random.rand(500, 110, 120)
print("Shape:", test_array.shape)
print("Size:", test_array.size)

Shape: (500, 110, 120)
Size: 6600000


In [12]:
# Run our function on this array
print(f"Result: {python_func(test_array):0.1f}")

# And then time it
%timeit python_func(test_array)
# On my computer this took around 1.35 seconds to run. Let's see how we can improve this.

Result: -1643400410.8
1.35 s ± 9.84 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [3]:
# The best advice for improving performance of Python when working with arrays of data is to use numpy instead.

def numpy_func(array_3d):

    # The sum of all the members of the array is easy to do in numpy:
    # sum = np.sum(array_3d)

    # But we need to subtract off the x-location for each value before we add it. Let's do that first.
    # Make a new array with x locations
    x_len, y_len, z_len = array_3d.shape
    x_locations = np.arange(0, x_len)

    # To perform the subtraction we can make use of Numpy's newaxis feature
    # Read more about it here: https://numpy.org/doc/stable/user/basics.indexing.html#dimensional-indexing-tools
    array_subtract = array_3d - x_locations[:, np.newaxis, np.newaxis]

    # Now we can perform the sum on the new subtracted 3D array
    result = np.sum(array_subtract)
    return result


In [11]:
# Test to see if we get the same answer as before
print(f"Numpy Result: {numpy_func(test_array):0.1f}")

# And then time it
%timeit numpy_func(test_array)
# On my computer this took around 25.4ms. Or about 53x faster

Numpy Result: -1643400410.8
25.4 ms ± 1.55 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [4]:
# That numpy function is great but we had to have known about the np.newaxis trick. 
# We may not always know about those tricks or they may not exist. What if we really need to use the method with 3 for loops?
# Well there is another tool we can turn to that does not require us to do any additional work (at least for our example here)

# Let us rewrite our function but this time apply the `numba` package's `njit` "decorator"
import numba as nb

@nb.njit
def numba_func(array_3d):

    # Get the size of each dimension
    x_len, y_len, z_len = array_3d.shape

    output = 0
    for i in range(x_len):
        for j in range(y_len):
            for k in range(z_len):

                # Output is a sum of each member of the array minus that item's x-location
                output += array_3d[i, j, k] - i
    return output

In [5]:
# Like before, let's see if we get the same answer
import time
t0_nba = time.time()
print(f"Numpy Result: {numba_func(test_array):0.1f}")
tf_nba = time.time()

# And then time it
%timeit numba_func(test_array)
# On my computer this took around 6.07ms. 220x faster than Python and almost 10x faster than numpy!

# "startup" costs
t0_python = time.time()
print(f"Result: {python_func(test_array):0.1f}")
t1_python = time.time()

print(f'\n\nRunning Python took {t1_python-t0_python:0.5f} seconds.')
print(f'The _first_ time numba was run it took {tf_nba-t0_nba:0.5f} seconds.')  # ~0.4 seconds compared to 6.07ms


Numpy Result: -1643398861.5
6.09 ms ± 149 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
Result: -1643398861.5
Running Python took 1.45048 seconds.
The _first_ time numba was run it took 0.40812 seconds.


Numba is a great solution for quickly speeding up python functions but there are some catches and limitations.
- It will always be slow (sometimes very slow) the _first_ time it is called on a new type of input.
- It does not work with a lot of 3rd party packages. 
    - It can be used in functions that use some numpy and scipy functions. Most other 3rd party packages are not supported.
- Occasionally it can be a pain to use and tricky to debug. 

Recommendation:
- If you have a function that performs simple operations on an array: Try to just use numpy tricks.
- If you can't figure out a numpy trick that works _AND_ your function does not depend on other 3rd party packages: try out numba.
- If your function does depend on other packages try to isolate parts that do not and wrap them in a numba function.

There are other performance tricks within python itself and other 3rd party packages (check out `Cython` as an alternative to numba).

## Debugging Python
Debugging is always _fun_. The best way to debug is to not debug at all. You would be surprised how much you can avoid by having proper unit tests (see later section). 

However, no matter how much you try to avoid it, you will have to debug eventually. 

In [6]:
# `print` debugging - Simply add print statements everywhere.

# Suppose there is a function that is not giving me the correct result

def broken_func(scale, b):

    if scale < 0 and b > 0:
        return b
    elif scale > 0 and b > 0:
        return b - scale
    else:
        return scale

print(broken_func(0, 3))  # I am expecting to get back 3 - 0 == 3; but I get back 0.


0


In [7]:
# I can add print commands everywhere to find the problem
def broken_func2(a, b):
    
    print(f'Func Start scale={scale}, b={b}')
    if scale < 0 and b > 0:
        print('Check 1')
        return b
    elif scale > 0 and b > 0:
        print('Check 2')
        return b - scale
    else:
        print('Check 3')
        return scale

print(broken_func2(0, 3))
# Now I can see that the code never reaches check 2. I see the bug is caused by "a > 0" should be "a >= 0"

Func Start a=0, b=3
Check 3
0


In [8]:
# We can also use Python's built in debugger "pdb.set_trace()" to create an interactive terminal for us to explore variables
# Note that newer versions of python allow you to replace "pdb.set_trace()" with simply "breakpoint", but this currently does not work in jupyter notebooks.

def broken_func3(scale, b):
    
    print(f'Func Start scale={scale}, b={b}')

    import pdb
    pdb.set_trace()

    if scale < 0 and b > 0:
        print('Check 1')
        return b
    elif scale > 0 and b > 0:
        print('Check 2')
        return b - scale
    else:
        print('Check 3')
        return scale

print(broken_func3(0, 3))

Func Start a=0, b=3
> c:\users\jrenaud\appdata\local\temp\1\ipykernel_18120\2001527067.py(11)broken_func3()

a = 0
b = 3
a = 0
b = 3
3


## Astropy Interactions

## Unit Testing

# Challenges